In [1]:
import traitlets
import ipywidgets as widgets
from containerized_gui.widgets import GUIContainer, FILE
from IPython.display import SVG
from os.path import basename
import base64
from IPython.display import HTML


class Data(traitlets.HasTraits):
    template = traitlets.Dict(traitlets.Unicode())
    output_files = traitlets.List(traitlets.Unicode())
    svg_snapshots = traitlets.List(traitlets.Unicode())
    
    @traitlets.observe('output_files')
    def _observer_output_files(self, change):
        print("_observer_output_files called", self.output_files)
        self.svg_snapshots = sorted(list(filter(lambda f: basename(f).startswith("snapshot") and f.endswith(".svg"), self.output_files)))
        
    @traitlets.observe('template')
    def _observer_template(self, change):
        print("_observer_template called")
        self.output_files = []
    
    
data = Data()

# Just for debugging
# out = widgets.Output()
# data.observe(lambda c: out.append_stdout(str(c) + "\n\n"))
# out

# PhysiCell Sample Projects

Select a PhysiCell sample project below. You'll then be able to edit and run that model.

In [2]:
template_dropdown = widgets.Dropdown(
    options=[('Select a sample', ''), 
             ('Biorobots (2D)', 'biorobots'), 
             ('Anti-cancer biorobots (2D)', 'cancer-biorobots'), 
             ('Template', 'template')],
    value='',
    description='Sample Project:',
)
def template_changed(change):
    if change['new'] == 'biorobots':
        data.template = {'template': "./data/physicell_biorobots.xml", 'exec': '/usr/local/bin/biorobots'}
    elif change['new'] == 'template':
        data.template = {'template': "./data/physicell_template.xml", "exec": "/usr/local/bin/project"}
    else:
        data.template = {}
template_dropdown.observe(template_changed, 'value')
template_dropdown.style.description_width = "100px"
template_dropdown

Dropdown(description='Sample Project:', options=(('Select a sample', ''), ('Biorobots (2D)', 'biorobots'), ('A…

In [3]:
hbox = widgets.HBox()
gui_container_widget = None
output_files_dlink = None
def data_template_changed(change):
    global gui_container_widget
    global output_files_dlink
    if not change.new.get('template'):
        return
    if gui_container_widget is not None:
        gui_container_widget.shutdown()
    if output_files_dlink is not None:
        output_files_dlink.unlink()
    print("starting gui container")
    gui_container_widget = GUIContainer(image_name="physicell", 
                                        width=1100, 
                                        input_file=data.template['template'], 
                                        args=["--studio", "--config", FILE, "-e", data.template['exec']])
    output_files_dlink = traitlets.dlink((gui_container_widget, 'output_files'), (data, 'output_files'))
    hbox.children = [gui_container_widget]

data.observe(data_template_changed, 'template')
hbox

HBox()

In [4]:
snapshot_slider = widgets.IntSlider(min=1, value=1, max=1, description="Snapshot Image" )
snapshot_slider.style.description_width = "100px"
snapshot_output = widgets.Output()
vbox = widgets.VBox([snapshot_slider, snapshot_output])

def svg_snapshots_changed(change):
    snapshot_slider.value = 1
    snapshot_slider.max = max(1, len(data.svg_snapshots))
    if len(data.svg_snapshots) > 0:
        selected_snapshot_changed()
    else:
        snapshot_output.outputs = ()
data.observe(svg_snapshots_changed, 'svg_snapshots')

def selected_snapshot_changed(change=None):
    snapshot_output.outputs = ()
    if snapshot_slider.value <= len(data.svg_snapshots):
        selected_snapshot = data.svg_snapshots[snapshot_slider.value - 1]
        with open(selected_snapshot, 'rb') as svg_file:
            svg_html = HTML(f'<img src="data:image/svg+xml;base64,{base64.b64encode(svg_file.read()).decode()}" style="width: 50%">')
            snapshot_output.append_display_data(svg_html)
snapshot_slider.observe(selected_snapshot_changed, 'value')

vbox

In [87]:
# snapshot_slider.max = len(data.svg_snapshots)